In [18]:

from lark import Lark, Tree
from transformer import SimaiTransformer
import subprocess
import sys

In [44]:
parser = Lark.open('../simai.lark', parser="lalr")
lark_data = parser.parse(open(f'../example/maidata.txt', encoding='UTF8').read())
metadata = SimaiTransformer().transform(lark_data)

In [47]:
title = None
artist = None
bpm = None
first = 0
charters = {}
for node in lark_data.children:
    if type(node) is not Tree:
        continue
    if node.data == 'artist':
        artist = str(node.children[0])
    if node.data == 'title':
        title = str(node.children[0])
    if node.data == 'wholebpm': # 10 bpm은 넘길 거임.
        bpm = float(str(node.children[0]))
    if node.data == 'first':
        first = float(str(node.children[0]))
    if node.data == 'des':
        if type(node.children[0]) is Tree:
            if len(node.children[0].children) > 0:
                for i in range(6):
                    charters[i] = str(node.children[0].children[0])
        else:
            charters[int(str(node.children[0])) - 2] =  str(node.children[1].children[0])
        

levels = {}
first_bpms = {}
for data in metadata:
    if data['type'] == 'level':
        levels[data['value'][0] - 2] = data['value'][1]
    if data['type'] == 'chart':
        first_bpms[data['value'][0] - 2] = data['value'][1].split('(')[1].split(')')[0]
        


print(charters.get(0))
print(first)

None
0.21


In [34]:
# QuiQ convert
initial_bpm = 120
first = 0.05

# calculate offset
one_bar_time = 60 / (initial_bpm / 4)
one_grid_time = one_bar_time / 384
offset = first / one_grid_time
print(one_bar_time)
print(one_grid_time)
print(offset)
print(round(offset))

2.0
0.005208333333333333
9.600000000000001
10


In [27]:
is_festival = True


level_dict = {
    '1': 1, '2': 2,
    '3': 3, '4': 4,
    '5': 5, '6': 6,
    '7': 7, '7+': 8,
    '8': 9, '8+': 10,
    '9': 11, '9+': 12,
    '10': 13, '10+': 14,
    '11': 15, '11+': 16,
    '12': 17, '12+': 18,
    '13': 19, '13+': 20,
    '14': 21, '14+': 22,
    '15': 23, '15+': 24
}

levels = {}
for data in metadata:
    if data['type'] == 'level':
        levels[data['value'][0] - 2] = data['value'][1]

notes = []
for i in range(6):
    if i not in levels:  # 채보 있는놈
        print(f'no level {i}')
        # notes.append({
        #     'file': {'path': f'{music_id}_0{i}.ma2'},
        #     'level': '0',
        #     'levelDecimal': '0',
        #     'notesDesigner': {'id': '0', 'str': None},
        #     'notesType': '0',
        #     'musicLevelID': '0',
        #     'maxNotes': '0',
        #     'isEnable': 'false'
        # })
    else:  # 채보 없는놈
        level = levels[i]

        leveldecimal = '0'
        if '+' in level:
            leveldecimal = '7'

        # 채보 변환 (result.ma2 생성)
        proc = subprocess.run(
            f'..\\MaichartConverter_Win1071\\MaichartConverter.exe CompileSimai -p ../example/maidata.txt -f {'Ma2_104' if is_festival else 'Ma2'} -o ./output -d {i + 2}',
            # ['.\\MaichartConverter_Win1071\\MaichartConverter.exe', 'CompileSimai', '-p', f'./{in_path}/maidata.txt', '-f', 'Ma2_104' if is_festival else 'Ma2', '-o', f'{out_path}/tmp_{music_id}', 'd', f'{i + 2}'],
            stdin=subprocess.DEVNULL, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, shell=True
        )
        print(str(proc.stdout))

        # offset 미루는 부분
        with open(f'./output/result.ma2', encoding='utf-8') as f:
            data = f.read()

        data = data.split('\n\n')
        new_chart = []
        for line in data[2].split('\n'):
            line = line.split('\t')
            total = int(line[1]) * 384 + int(line[2])
            line[1] = str(total // 384)
            line[2] = str(total % 384)
            new_chart.append('\t'.join(line))
        data[2] = '\n'.join(new_chart)
        data = '\n\n'.join(data)

        # with open(f'{out_path}/music/music{music_id}/{music_id}_0{i}.ma2', 'w', encoding='utf-8') as f:
        #     f.write(data)
        # 
        # # metadata 작성
        # notes.append({
        #     'file': {'path': f'{music_id}_0{i}.ma2'},
        #     'level': level.replace('+', ''),
        #     'levelDecimal': leveldecimal,
        #     'notesDesigner': {'id': '0', 'str': 'NZN'},
        #     'notesType': '0',
        #     'musicLevelID': str(level_dict[level]),
        #     'maxNotes': '0',
        #     'isEnable': 'true'
        # })

no level 0
no level 1
no level 2
no level 3
no level 4
b'     _____         .__       .__                   __   _________                                   __                \n    /     \\ _____  |__| ____ |  |__ _____ ________/  |_ \\_   ___ \\  ____   _______  __ ____________/  |_  ___________ \n   /  \\ /  \\\\__  \\ |  |/ ___\\|  |  \\\\__  \\\\_  __ \\   __\\/    \\  \\/ /  _ \\ /    \\  \\/ // __ \\_  __ \\   __\\/ __ \\_  __ \\\n  /    Y    \\/ __ \\|  \\  \\___|   Y  \\/ __ \\|  | \\/|  |  \\     \\___(  <_> )   |  \\   /\\  ___/|  | \\/|  | \\  ___/|  | \\/\n  \\____|__  (____  /__|\\___  >___|  (____  /__|   |__|   \\______  /\\____/|___|  /\\_/  \\___  >__|   |__|  \\___  >__|   \n          \\/     \\/        \\/     \\/     \\/                     \\/            \\/          \\/                 \\/       \na GUtils component for rhythm games\nRev 1.0.7.1 by Neskol\nCheck https://github.com/Neskol/MaichartConverter for updates and instructions\n\r\n\r\nExecuting CompileSima

In [23]:
param1_list = list(range(20))
param2_list = list(range(20, 40))
param3_list = list(range(40, 60))
param4_list = list(range(60, 80))
param5_list = list(range(80, 100))

# 파라미터 튜플로 묶기
params_list = list(zip(param1_list, param2_list, param3_list, param4_list, param5_list))

In [24]:
params_list

[(0, 20, 40, 60, 80),
 (1, 21, 41, 61, 81),
 (2, 22, 42, 62, 82),
 (3, 23, 43, 63, 83),
 (4, 24, 44, 64, 84),
 (5, 25, 45, 65, 85),
 (6, 26, 46, 66, 86),
 (7, 27, 47, 67, 87),
 (8, 28, 48, 68, 88),
 (9, 29, 49, 69, 89),
 (10, 30, 50, 70, 90),
 (11, 31, 51, 71, 91),
 (12, 32, 52, 72, 92),
 (13, 33, 53, 73, 93),
 (14, 34, 54, 74, 94),
 (15, 35, 55, 75, 95),
 (16, 36, 56, 76, 96),
 (17, 37, 57, 77, 97),
 (18, 38, 58, 78, 98),
 (19, 39, 59, 79, 99)]